# Setting Up All Artifacts details

In [1]:
import os
os.environ['QNN_SDK_ROOT']="/local/mnt/workspace/gokul/QNN/2.19.0.240124" #set up your qnn path here.
os.environ['ANDROID_NDK_ROOT']="/local/mnt/workspace/gokul/android-ndk-r25c"
os.environ['RAW_FILE_FOLDER']="raw"
os.environ['FOLDER_WITH_ARTIFACTS']="StableLLVE"
os.environ['ONDEVICE_FOLDER']="StableLLVE"
os.environ['MODEL_NAME']="StableLLVE_fp32"
os.environ['QUANTIZED_MODEL_NAME']="StableLLVE_w8a8"
os.environ['MODEL_PATH']="models/StableLLVE_fp32"
os.environ['QUANTIZED_MODEL_PATH']="models/StableLLVE_w8a8"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="bc468c1d" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['QNN_TARGET_ARCH']="aarch64-android"
os.environ['QNN_TARGET_STL']="libQnnHtp.so"

In [2]:
%%bash
git clone https://github.com/zkawfanx/StableLLVE.git

In [3]:
%%bash
cp generate_model.py StableLLVE/
cd StableLLVE
python generate_model.py

/local/mnt/workspace/gokul/gokul-env/lib/python3.8/site-packages/torch/onnx/utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/local/mnt/workspace/gokul/gokul-env/lib/python3.8/site-packages/torch/onnx/utils.py:1209: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(


ONNX model saved Successfully


#### Getting the Non Quantized DLC File

#### Understanding the Architecture of the Model

Use the dataset of your choice to validate the pre-processing, and post processing steps given in this notebook

In [4]:
def preprocess(ll_img, hl_img):
    ll_img = np.array(ll_img, dtype='float32')
    hl_img = np.array(hl_img, dtype='float32')   
    ll_img = np.uint8(np.clip(ll_img, 0., 255.))  # this is to simulate matlab's imwrite operation
    hl_img = np.uint8(np.clip(hl_img, 0., 255.))  # this is to simulate matlab's imwrite operation 
    ll_img = np.uint8(ll_img)
    hl_img = np.uint8(hl_img)
    ll_img = convert_image(ll_img, source='array', target='[0, 1]')
    hl_img = convert_image(hl_img, source='array', target='[0, 1]')
    return ll_img, hl_img

## Preprocessing the data to generate raw file

In [5]:
def convert_image(img, source, target):
    if source == 'array':
        img = torch.from_numpy(img.transpose((2, 0, 1))).contiguous()#chw
        img = img.to(dtype=torch.float32).div(255) 
    elif source == '[0, 1]':
        img = torch.clamp(img, 0, 1)  # useful to post-process output of models that can overspill
    
    if target == '[0, 1]':
        pass  # already in [0, 1]
    return img

In [6]:
def post_process(img):
    img = img.detach().cpu().numpy()
    img = np.clip(255. * img, 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img

In [7]:
def load_dataset(test_images_dir):
    # Input images for the model
    INPUTS_LL = []
    # Post-processed images for visualization
    IMAGES_LL = [] # LL:Low Light
    IMAGES_HL = [] # HL:High Light

    # Load the test images
    count=0
    for img_path in glob.glob(os.path.join(test_images_dir, '*')):
        l_img = cv2.resize(cv2.imread(img_path),[480,640],interpolation=cv2.INTER_CUBIC)
        l_img = cv2.cvtColor(l_img, cv2.COLOR_BGR2RGB)
        
        h_img = cv2.resize(cv2.imread(img_path.replace("low","high")),[480,640],interpolation=cv2.INTER_CUBIC)
        h_img = cv2.cvtColor(h_img, cv2.COLOR_BGR2RGB)
        
        ll_img, hl_img = preprocess(l_img, h_img)#chw

        INPUTS_LL.append(ll_img)#chw
        IMAGES_LL.append(post_process(ll_img))#hwc
        IMAGES_HL.append(post_process(hl_img))#hwc
    return INPUTS_LL, IMAGES_LL, IMAGES_HL

**Converting the low dataset to raw file format to give it to the model and do the inference**

In [8]:
os.makedirs('raw', exist_ok=True)

In [9]:
import glob
import os
import cv2
import numpy as np
import torch

test_images_dir = "eval15/low"
INPUTS_LL, IMAGES_LL, IMAGES_HL = load_dataset(test_images_dir)
print(len(INPUTS_LL),len(IMAGES_LL),len(IMAGES_HL))
for i, img_ll in enumerate(INPUTS_LL):
    img_ll = img_ll.cpu().detach().numpy()
    img_ll = img_ll.astype(np.float32)
    fid = open("raw/img_"+str(i)+ ".raw", 'wb')
    img_ll.tofile(fid)
    fid.close()

0 0 0


In [10]:
def post_process_enhanced(img):
    img = np.fromfile(img, np.float32)
    img = img.reshape((3,640,480)).astype(np.float32)
    img = np.clip(img* 255. , 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img

#### Getting the Quantized Model

In [11]:
total_iter = 15
print("Generating input_list \"small_raw_list.txt\" with {} iterations".format(total_iter))
with open("list.txt",'w') as f:
    for i in range(total_iter):
        f.write("raw/img_{}.raw\n".format(i))

Generating input_list "small_raw_list.txt" with 15 iterations


In [12]:
%%bash
source ${QNN_SDK_ROOT}/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-onnx-converter --input_network StableLLVE.onnx --output_path ${MODEL_PATH}.cpp
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-model-lib-generator -c ${MODEL_PATH}.cpp -b ${MODEL_PATH}.bin -o models/model_libs
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-context-binary-generator \
              --backend ${QNN_SDK_ROOT}/lib/x86_64-linux-clang/libQnnHtp.so \
              --model models/model_libs/x86_64-linux-clang/lib${MODEL_NAME}.so \
              --binary_file ${MODEL_NAME}.serialized

In [13]:
%%bash
source ${QNN_SDK_ROOT}/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-onnx-converter --input_network StableLLVE.onnx --output_path ${QUANTIZED_MODEL_PATH}.cpp --input_list list.txt \
                --param_quantizer "adjusted" --act_quantizer "enhanced"
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-model-lib-generator -c ${QUANTIZED_MODEL_PATH}.cpp -b ${QUANTIZED_MODEL_PATH}.bin -o models/model_libs2
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-context-binary-generator \
              --backend ${QNN_SDK_ROOT}/lib/x86_64-linux-clang/libQnnHtp.so \
              --model models/model_libs2/x86_64-linux-clang/lib${QUANTIZED_MODEL_NAME}.so \
              --binary_file ${QUANTIZED_MODEL_NAME}.serialized

[INFO] AISW SDK environment set
[INFO] QNN_SDK_ROOT: /local/mnt/workspace/gokul/QNN/2.19.0.240124


2024-03-04 09:44:17,228 - 235 - INFO - Simplified model validation is successful
2024-03-04 09:44:18,444 - 235 - INFO - Saving QNN Model...


IrQuantizer: Quantizer param type: adjusted will be deprecated in future releases
IrQuantizer: Quantizer type: adjusted is no longer supported. Using TF quantizer instead


2024-03-04 09:44:37,127 - 235 - INFO - Model CPP saved at: models/StableLLVE_w8a8.cpp 
2024-03-04 09:44:37,128 - 235 - INFO - Model BIN saved at: /local/mnt/workspace/gokul/models-for-solutions/02-low-light-enhancement/StableLLVE/models/StableLLVE_w8a8.bin 
2024-03-04 09:44:37,128 - 235 - INFO - Conversion complete!


     0.1ms [  INFO ] Inferences will run in sync mode
     0.1ms [  INFO ] Initializing logging in the backend. Callback: [0x7fb5d8ff2e10], Log Level: [3]
     0.1ms [  INFO ] No BackendExtensions lib provided;initializing NetRunBackend Interface
     0.2ms [WARNING] Unable to find a device with NetRunDeviceKeyDefault in Library NetRunBackendLibKeyDefault
     0.2ms [  INFO ] Entering QuantizeRuntimeApp flow
     4.3ms [  INFO ] CpuGraph::finalize
   288.8ms [  INFO ] CpuGraph::execute
  1794.6ms [  INFO ] cleaning up resources for input tensors
  1794.6ms [  INFO ] cleaning up resources for output tensors
  1819.3ms [  INFO ] CpuGraph::execute
  2959.3ms [  INFO ] cleaning up resources for input tensors
  2959.3ms [  INFO ] cleaning up resources for output tensors
  2981.4ms [  INFO ] CpuGraph::execute
  4153.4ms [  INFO ] cleaning up resources for input tensors
  4153.5ms [  INFO ] cleaning up resources for output tensors
  4171.1ms [  INFO ] CpuGraph::execute
  5378.5ms [  INFO ] cl

jni/StableLLVE_w8a8.cpp:39:44: warning: mixture of designated and non-designated initializers in the same initializer list is a C99 extension [-Wc99-designator]
                                           .id=0,
                                           ^~~~~
jni/QnnWrapperUtils.hpp:77:17: note: expanded from macro 'VALIDATE'
    retStatus = value;                                                                           \
                ^~~~~
jni/StableLLVE_w8a8.cpp:50:44: note: first non-designated initializer is here
                                           {.clientBuf= { .data=nullptr,
                                           ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
jni/QnnWrapperUtils.hpp:77:17: note: expanded from macro 'VALIDATE'
    retStatus = value;                                                                           \
                ^~~~~
jni/StableLLVE_w8a8.cpp:37:42: warning: mixture of designated and non-designated initializers in the same initializer list is a C99 extens

/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/down1_maxpool_conv_1_double_conv_0_bias.raw obj/local/arm64-v8a/objs/StableLLVE_w8a8//down1_maxpool_conv_1_double_conv_0_bias.o
/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/down1_maxpool_conv_1_double_conv_0_weight.raw obj/local/arm64-v8a/objs/StableLLVE_w8a8//down1_maxpool_conv_1_double_conv_0_weight.o
/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/down1_maxpool_conv_1_double_conv_2_bias.raw obj/local/arm64-v8a/objs/StableLLVE_w8a8//down1_maxpool_conv_1_double_conv_2_bias.o
/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/bin

## Model Inference

In [14]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/$QNN_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/$QNN_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/dsp/lib"

In [15]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [16]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/$QNN_TARGET_ARCH/$QNN_TARGET_STL /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/bin/$QNN_TARGET_ARCH/qnn-net-run /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/hexagon-v73/unsigned/*.so /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/$QNN_TARGET_ARCH/*.so /data/local/tmp/$ONDEVICE_FOLDER

/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/aarch64-android/libQnnHtp.so: 1 file pushed, 0 skipped. 405.5 MB/s (1545520 bytes in 0.004s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/bin/aarch64-android/qnn-net-run: 1 file pushed, 0 skipped. 408.6 MB/s (1441208 bytes in 0.003s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libCalculator_skel.so: 1 file pushed, 0 skipped. 150.5 MB/s (7128 bytes in 0.000s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnHtpV73QemuDriver.so: 1 file pushed, 0 skipped. 495.7 MB/s (7601584 bytes in 0.015s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnHtpV73Skel.so: 1 file pushed, 0 skipped. 511.7 MB/s (7360784 bytes in 0.014s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnHtpV73.so: 1 file pushed, 0 skipped. 502.9 MB/s (8025008 bytes in 0.015s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnSaver.so: 1 file pushed,

In [17]:
%%bash
#find ./raw -name *.raw > list.txt
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push output/${QUANTIZED_MODEL_NAME}.serialized.bin /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push output/${MODEL_NAME}.serialized.bin /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push models/model_libs2/aarch64-android/lib${QUANTIZED_MODEL_NAME}.so /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

output/StableLLVE_w8a8.serialized.bin: 1 file pushed, 0 skipped. 748.1 MB/s (4856800 bytes in 0.006s)
output/StableLLVE_fp32.serialized.bin: 1 file pushed, 0 skipped. 226.8 MB/s (17633544 bytes in 0.074s)
models/model_libs2/aarch64-android/libStableLLVE_w8a8.so: 1 file pushed, 0 skipped. 489.0 MB/s (4714928 bytes in 0.009s)
raw/: 15 files pushed, 0 skipped. 305.5 MB/s (55296000 bytes in 0.173s)
list.txt: 1 file pushed, 0 skipped. 5.2 MB/s (215 bytes in 0.000s)


#### Inferencing the Quantized Model

In [18]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export ONDEVICE_FOLDER="StableLLVE"
export device_path=/data/local/tmp/$ONDEVICE_FOLDER
export LD_LIBRARY_PATH=$device_path
export ADSP_LIBRARY_PATH="$device_path"
export PATH=$PATH:$device_path
export QUANTIZED_MODEL_NAME="StableLLVE_w8a8"
cd $device_path
export OUTPUT_FOLDER=OUTPUT_Quant_DSP
cd /data/local/tmp/$ONDEVICE_FOLDER  && 
./qnn-net-run --backend libQnnHtp.so --input_list list.txt --retrieve_context ${QUANTIZED_MODEL_NAME}.serialized.bin --output_dir $OUTPUT_FOLDER

qnn-net-run pid:13074


#### Inferencing the Non-Quantized Model

In [19]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export ONDEVICE_FOLDER="StableLLVE"
export device_path=/data/local/tmp/$ONDEVICE_FOLDER
export LD_LIBRARY_PATH=$device_path
export ADSP_LIBRARY_PATH="$device_path"
export PATH=$PATH:$device_path
export QUANTIZED_MODEL_NAME="StableLLVE_w8a8"
cd $device_path
export OUTPUT_FOLDER=OUTPUT_CPU
cd /data/local/tmp/$ONDEVICE_FOLDER  && 
./qnn-net-run --backend libQnnCpu.so --input_list list.txt --model lib${QUANTIZED_MODEL_NAME}.so --output_dir $OUTPUT_FOLDER

qnn-net-run pid:13081


In [20]:
%%bash
rm -rf OUTPUT_Quant_DSP
rm -rf OUTPUT_CPU

#### Pulling the Output

In [21]:
%%bash

export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_Quant_DSP output/OUTPUT_Quant_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_CPU output/OUTPUT_CPU

/data/local/tmp/StableLLVE/OUTPUT_Quant_DSP/: 15 files pulled, 0 skipped. 127.6 MB/s (55296000 bytes in 0.413s)
/data/local/tmp/StableLLVE/OUTPUT_CPU/: 15 files pulled, 0 skipped. 171.2 MB/s (55296000 bytes in 0.308s)


### Post Processing and Calculating PSNR Value

In [22]:
import math
def compute_psnr(img_pred, img_true, data_range=255., eps=1e-8):
    err = (img_pred - img_true) ** 2 
    err = np.mean(err)
    return 10. * math.log10((data_range ** 2) / (err + eps))

In [23]:
def evaluate_average_psnr(el_images, hl_images): #(enhanced_light<model o/p>, high_light <groud_truth>)
    psnr = []
    for i in range(len(el_images)):
        el_img = cv2.imread(el_images[i], 1)
        hl_img = cv2.imread(hl_images[i], 1)
        psnr.append(compute_psnr(el_img,hl_img))
    average_psnr = np.mean(np.array(psnr))
    return average_psnr